## Test baseline accuracy MATLAB VS Python data 

In [2]:
%%time
from model_set import EEGNet, SugiyamaNet
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from scipy.io import loadmat
import matplotlib.pyplot as plt
from scipy.signal import lfilter, butter

import capilab_dataset2
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold

ALL_SUBJS = ['Takahashi', 'Lai', 'Sugiyama', 'Suguro']


def load1(subj):    
    fs = 500
    duration = 2
    sample = fs * duration
    ch = 19
    hp = 0.5
    lp = 40
    #data1, label1 = capilab_dataset2.load_target('Takahashi_JulyData')
    #data2, label2 = capilab_dataset2.load_target('Suguro_JulyData')
    #data3, label3 = capilab_dataset2.load_target('Lai_JulyData')
    #data4, label4 = capilab_dataset2.load_target('Sugiyama_JulyData')
    #data = np.vstack([data1, data2, data3, data4])
    data, label = capilab_dataset2.load_target(subj + '_JulyData')
    #label = np.vstack([label1, label2, label3, label4])
    try:
        x, x_test,  y, y_test = train_test_split(data, label, test_size = .2, stratify = label, random_state = 1)
        x = capilab_dataset2.butterworth_bpf(x, hp, lp, fs)
        x_test = capilab_dataset2.butterworth_bpf(x_test, hp, lp, fs)
        x = np.expand_dims(x, axis = 3)
        x_test = np.expand_dims(x_test, axis = 3)
        # # swap sample and channels axis
        # x = np.transpose(x, (0,2,1,3))
        # x_test = np.transpose(x_test, (0,2,1,3))
        
    except Exception as ex:
        template = "An exception of type {0} occurred. Arguments:\n{1!r}"
        message = template.format(type(ex).__name__, ex.args)
        print(message)
        return None
    else:
        return x, y, x_test, y_test

def load2(target_file):    

    try:
        contents= loadmat(target_file)
        X = contents['raw_x']
        Y = contents['raw_y']

        #target shape = (data, 19, 1000, 1), from (1000, 19, data)
        X = np.transpose(X, [2,1,0])
        Y = np.transpose(Y, [1,0])
        X = np.expand_dims(X, axis = 3)
        x, x_test,  y, y_test = train_test_split(X, Y, test_size = .1, stratify = Y, random_state = 1)
    except Exception as ex:
        template = "An exception of type {0} occurred. Arguments:\n{1!r}"
        message = template.format(type(ex).__name__, ex.args)
        print(message)
        return None
    else:
        return x, y, x_test, y_test
    
def _step(X, y, x_val, y_val, x_test, y_test, verbose = False):
    tf.keras.utils.set_random_seed(1)
    tf.config.experimental.enable_op_determinism()
    classifier_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    classifier_loss = tf.keras.losses.CategoricalCrossentropy()
    
    clf = SugiyamaNet(Chans = X.shape[1], Samples = X.shape[2], output_classes = y.shape[1])
    # clf = EEGNet(y.shape[1], Chans = X.shape[1], Samples = X.shape[2], 
                                            # dropoutRate = 0.5, kernLength = 512, F1 = 64, 
                                            # D = 8, F2 = 128, norm_rate = 0.25, dropoutType = 'Dropout')
    
    clf.compile(optimizer = classifier_optimizer, loss= classifier_loss , metrics=['accuracy'])
    
    checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath='/tmp/checkpoint.h5', verbose=False, save_best_only=True)
    earlystopper = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.001, patience=4, verbose=0)
    clf.fit(X, y,
            batch_size=8, 
            epochs = 12, 
            verbose = verbose, 
            validation_data = (x_val, y_val),
            callbacks = [checkpointer, earlystopper])
    y_preds = clf.predict(x_test, verbose = verbose)
    predicted = np.argmax(y_preds, axis=1)
    ground_truth = np.argmax(y_test, axis=1)
    
    r = accuracy_score(ground_truth, predicted)
    # clf.save('temp_model')
    return r

def __train1(subj_name = None, b = 8, ep = 12):


    f1 = []
    acc = []
    x, y, x_test,y_test = load1(subj_name) #DATA

    # print(x.shape)  
    r = np.array([])
    kfold = StratifiedKFold(n_splits = 4, shuffle = True, random_state = 0) #fold
    for i , (train, val) in enumerate(kfold.split(x, np.argmax(y, axis = 1))):
        # tf.random.set_seed(0)
        res = _step(x[train], y[train], x[val], y[val],x_test, y_test)
        r = np.append(r, res)
    
    tf.keras.backend.clear_session()
    return r.mean(), r.std(), r

def __train2(subj_name = None, b = 8, ep = 12):


    f1 = []
    acc = []
    x, y, x_test,y_test = load2(subj_name) #DATA

    # print(x.shape)  
    r = np.array([])
    kfold = StratifiedKFold(n_splits = 4, shuffle = True, random_state = 0) #fold
    for i , (train, val) in enumerate(kfold.split(x, np.argmax(y, axis = 1))):
        # tf.random.set_seed(0)
        res = _step(x[train], y[train], x[val], y[val],x_test, y_test)
        r = np.append(r, res)
    
    tf.keras.backend.clear_session()
    return r.mean(), r.std(), r


print("--------------------TKH---------------")
print(f'-Result of Python Filter  : {__train1("Takahashi")}')
print(f'-Result of Matlab Filter  : {__train2("Datasets/tkh_filtered_data.mat")}')
print("--------------------SGYM---------------")
print(f'-Result of Python Filter  : {__train1("Sugiyama")}')
print(f'-Result of Matlab Filter  : {__train2("Datasets/sgym_filtered_data.mat")}')
print("--------------------LYF---------------")
print(f'-Result of Python Filter  : {__train1("Lai")}')
print(f'-Result of Matlab Filter  : {__train2("Datasets/lyf_filtered_data.mat")}')
print("--------------------SGR---------------")
print(f'-Result of Python Filter  : {__train1("Suguro")}')
print(f'-Result of Matlab Filter  : {__train2("Datasets/sgr_filtered_data.mat")}')


--------------------TKH---------------
-Result of Python Filter  : (0.6187499999999999, 0.054126587736527385, array([0.58333333, 0.7       , 0.55833333, 0.63333333]))
-Result of Matlab Filter  : (0.6041666666666666, 0.06495190528383289, array([0.66666667, 0.65      , 0.6       , 0.5       ]))
--------------------SGYM---------------
-Result of Python Filter  : (0.6291666666666667, 0.037960139913101244, array([0.66666667, 0.56666667, 0.63333333, 0.65      ]))
-Result of Matlab Filter  : (0.6458333333333333, 0.032004773949452545, array([0.6       , 0.63333333, 0.66666667, 0.68333333]))
--------------------LYF---------------
-Result of Python Filter  : (0.5125, 0.037499999999999985, array([0.575     , 0.50833333, 0.48333333, 0.48333333]))
-Result of Matlab Filter  : (0.5708333333333333, 0.04768967975941498, array([0.63333333, 0.58333333, 0.56666667, 0.5       ]))
--------------------SGR---------------
-Result of Python Filter  : (0.4604166666666667, 0.05962120008855912, array([0.475     , 

In [ ]:
#test filter
from scipy.io import loadmat
import matplotlib.pyplot as plt
from scipy.signal import lfilter, butter
fs = 500
durations = 2
sample = fs * durations
ch = 19

def butterworth_bandpass(low, high, fs, order = 4):
    return butter(order, [low, high], fs = fs, btype = 'band', output = 'ba', analog = False)

def butterworth_bpf(data, low, high, fs, order = 4):
    b,a = butterworth_bandpass(low, high, fs, order)
    return lfilter(b,a, data, axis = 1)

f = 'Datasets/targets.mat'
contents = loadmat(f)

target1 = contents['target1'] #1000 x 19 correct 
target2 = contents['target2'] #19 x 1000 incorrect
matlab_filt = np.transpose(contents['filtered'], [1,0])
filtered = butterworth_bpf(target2, 0.4, 40, 500 /2)

print(target1.shape, target2.shape, matlab_filt.shape, filtered.shape)

sim = np.subtract(filtered, matlab_filt)



plt.rcParams["figure.figsize"] = (15,10)
fig, ax = plt.subplots(4)

ax[0].set_title("raw")
ax[0].plot(target1)

ax[1].set_title("python filter")
ax[1].plot(filtered.transpose())

ax[2].set_title("matlab filter")
ax[2].plot(matlab_filt.transpose())

ax[3].set_title("diff")
ax[3].plot(sim.transpose())

plt.show()

print()


In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.metrics import mean_squared_error
from tensorflow.keras.losses import Huber

#test weigth copy
train_model = Sequential([
    Flatten(),             
    Dense(units=32,activation = 'relu'),
    Dense(units=64,activation = 'relu'),
    Dense(units=128,activation = 'relu'),
    Dense(4, activation = 'softmax')
])
train_model.compile(loss="huber", optimizer = Adam(learning_rate = 1e-3))
train_model.build((1,19))
target_model = Sequential([
    Flatten(),             
    Dense(units=32,activation = 'relu'),
    Dense(units=64,activation = 'relu'),
    Dense(units=128,activation = 'relu'),
    Dense(4, activation = 'softmax')
])
target_model.compile(loss="huber", optimizer = Adam(learning_rate = 1e-3))
target_model.build((1,19))

target_model.set_weights(train_model.get_weights())

In [16]:
from tqdm import tqdm
import time
total_steps = 0
good_exp = 0
bad_exp = 0

with tqdm(total = 1000, position = 0, leave = True) as pbar:
    for e in tqdm(range(1000), ncols = 100, position = 0, leave = True, desc ="DQN Training>"):
        for i in range(11):
            if e % 69 == 0:
                good_exp += 1
            else:
                bad_exp +=1
            time.sleep(0.1)
        
        if good_exp > 69 and bad_exp > 69:
            #train
            if total_steps %20 == 0:
                tqdm.write("Train di wa")    
            
        total_steps += 1
        pbar.update()

DQN Training>:  42%|██████████████████▉                          | 421/1000 [07:44<10:40,  1.11s/it]

Train di wa


DQN Training>:  44%|███████████████████▊                         | 441/1000 [08:06<10:18,  1.11s/it]

Train di wa


DQN Training>:  46%|████████████████████▋                        | 461/1000 [08:28<09:56,  1.11s/it]

...

DQN Training>:  96%|███████████████████████████████████████████▏ | 961/1000 [17:41<00:43,  1.11s/it]

Train di wa


DQN Training>:  98%|████████████████████████████████████████████▏| 981/1000 [18:03<00:21,  1.11s/it]

Train di wa


100%|██████████| 1000/1000 [18:24<00:00,  1.10s/it]
